# 1. Data Preparation

In [1]:
# Import Libraries Used in Data Preparation
import pandas as pd
import os
debug = True
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing


#### 1.1 Load Data from files

In [2]:
#Code to Load Data in a DataFrame
datafolder = '../data/'
#print(os.listdir(datafolder))
train_features_file_name = 'train_features.csv'
train_labels_file_name = 'train_salaries.csv'
test_features_file_name = 'test_features.csv'

# with open(os.path.join(datafolder, test_features_file_name)) as f:
#     x = f.readlines()
# print(x[1])
train_data_features = pd.read_csv(os.path.join(datafolder, train_features_file_name))
org_features = train_data_features.columns.tolist()
train_data_labels = pd.read_csv(os.path.join(datafolder, train_labels_file_name))
train_full = pd.merge(train_data_features, train_data_labels, on='jobId',how ='inner')
test_data_features = pd.read_csv(os.path.join(datafolder, test_features_file_name))

assert(train_full.shape[0] == train_data_features.shape[0])
assert(train_full.shape[0] == train_data_labels.shape[0])
#train_full.dropna(how ='any', inplace = True)
train_full.shape

(1000000, 9)

In [3]:
# 1000000
# 1000000
org_features

['jobId',
 'companyId',
 'jobType',
 'degree',
 'major',
 'industry',
 'yearsExperience',
 'milesFromMetropolis']

#### 1.2 Clean/Check Data (Pending)

In [4]:
#train_full.describe()
train_full.isnull().sum()
train_full_encoded = train_full
test_data_features_encoded= test_data_features

####  1.3 Encode Categorical Variables

1.3.1 One Hot Encoding : Done for *companyId*, *major*, and *industry* as these variabes are categories and may influence the salary

In [5]:
#Encode Categorical Variables One Hot

def one_hot_encoding(c, df):
    new_f_name = 'OH_'+ c
    df = pd.concat([df, pd.get_dummies(df[c], prefix= new_f_name)], axis=1)
    return df, new_f_name

cat_features_normal = ['companyId', 'major', 'industry']
cat_features_normal_new = []
for c in cat_features_normal:
    train_full_encoded, new_f_name = one_hot_encoding(c, train_full_encoded)
    test_data_features_encoded, new_f_name_test = one_hot_encoding(c, test_data_features_encoded)
    cat_features_normal_new.append(new_f_name)
    
    
print(cat_features_normal_new)
#train_full.head(1)

['OH_companyId', 'OH_major', 'OH_industry']


1.3.2 Ordinal  Encoding : Done for *degree* and *jobType* as these variabes have an inherent order associated with them

In [6]:
#Pending Visualize data to decide order

In [7]:
#Encode Categorical Variables Ordered

def ordinal_encoding(c, df, lookup):
    new_f_name = 'ORD_'+ c
    df[new_f_name] =  df[c].map(lookup)
    return df, new_f_name

degreeOrder = {'BACHELORS':2, 'DOCTORAL':4, 'HIGH_SCHOOL':1, 'MASTERS':3, 'NONE':0}
jobTypeOrder = {'CEO':7, 'CFO':6, 'CTO':5, 'VICE_PRESIDENT':4, 'MANAGER':3,'SENIOR':2, 'JUNIOR':1, 'JANITOR':0}

cat_features_ordinal = ['jobType', 'degree']
cat_features_ordinal_new = []
for c in cat_features_ordinal:
    train_full_encoded, new_f_name = ordinal_encoding(c, train_full_encoded, eval(c+'Order'))
    test_data_features_encoded, new_f_name_test = ordinal_encoding(c, test_data_features_encoded, eval(c+'Order'))
    cat_features_ordinal_new.append(new_f_name)

print(cat_features_ordinal_new)
#train_full.head(1)

['ORD_jobType', 'ORD_degree']


In [8]:
all_categorical_features = cat_features_ordinal + cat_features_normal_new
all_categorical_features

['jobType', 'degree', 'OH_companyId', 'OH_major', 'OH_industry']

In [9]:
# for c in cat_features_normal:
#     print(c)
#     test_data_features_encoded, new_f_name_test = one_hot_encoding(c, test_data_features_encoded)

# for c in cat_features_ordinal:
#     test_data_features_encoded, new_f_name_test = ordinal_encoding(c, test_data_features_encoded, eval(c+'Order'))

#print(len(train_full_encoded.columns))
#print(len(test_data_features_encoded.columns))
#train_full_encoded.columns

#### 1.4 Shuffle Data and Split into K folds which are used during model selection

In [10]:
#Shuffle Train Data
train_full_encoded =  train_full_encoded.sample(frac=1).reset_index(drop=True)

In [11]:
# Split Data into K Folds
from sklearn.model_selection import KFold
def GetKFoldData(df,k):
    folds = {}
    kfolds = KFold(n_splits=k, shuffle = True, random_state = 4) 
    foldidx = 0
    for train_idx, test_idx in kfolds.split(df.index):
        folds[foldidx] = { 'train': df.iloc[train_idx], 'test':df.iloc[test_idx]}
        foldidx+=1
    return folds

#exp_data_k_folds = GetKFoldData(train_full, 10)
#idx_list_org = exp_data_k_folds[0]['test'].index.tolist()
#exp_data_k_folds[0]['test'].shape


In [12]:
# set(test_data_features.jobType.tolist())
# set(test_data_features.degree.tolist())

In [13]:
scalermain = preprocessing.StandardScaler()
#scalermain = preprocessing.MinMaxScaler()

# 2. Model Selection

#### 2.1 Decide Features to Use and Models to Evaluate (Feature Scaling Pending)

2.1.1 Features to be Used (Filtered out encoded variables)

In [14]:
# Select Features
non_cat_features = ['yearsExperience', 'milesFromMetropolis']
cat_features =[]
for f in train_full_encoded.columns:
    if 'OH' in f or 'ORD' in f:
        cat_features.append(f)
        
#train_full[train_features].head()
all_features = non_cat_features + cat_features 
features_to_scale = non_cat_features + cat_features_ordinal_new
#train_features

2.1.2 Regression Models Considered

In [15]:
# Different Regression Models Evaluated
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor


models = {
            'LR' :    {'modelobj' :  linear_model.LinearRegression(), 'mse':None, 'rmse':None,\
                                     'tr_time':None,'tst_time':None},
    
            'Ridge' : {'modelobj' :  linear_model.Ridge(), 'mse':None, 'rmse':None,\
                                     'tr_time':None,'tst_time':None,\
                                     'params' : {'alpha': [i for i in range(1,81,3)]}},
    
            'DT' :    {'modelobj' :  DecisionTreeRegressor(random_state=10), 'mse':None, 'rmse':None, \
                                     'tr_time':None,'tst_time':None, \
                                     'params' : {
                                                 'max_depth':[i for i in range(9,12)], \
                                                 'min_samples_leaf' :[i for i in range(10,21,5)]
                                                }}#,   
    
#             'RF' :    {'modelobj' :  RandomForestRegressor(random_state=10), 'mse':None, 'rmse':None, \
#                                      'tr_time':None,'tst_time':None, \
#                                      'params' : {
#                                                  'n_estimators':[i for i in range(300,501,100)],
#                                                  'max_depth':[i for i in range(9,12)], 
#                                                  'min_samples_leaf' :[i for i in range(10,21,5)],
#                                                  'max_features' : ['log2', 'sqrt']
#                                                  }}
        
            #'GBR' : {'modelobj' :  GradientBoostingRegressor(), 'mse':None, 'rmse':None}
         
         }


#### 2.2 Select Model

In [16]:
# Helper Functions to Evaluate a Model given train and test data
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
import time

def EvalModelSplitData(m, model, curr_fold, all_features, features_to_scale, \
                       scaler = preprocessing.StandardScaler()):
    
    #Train Data Selection
    curr_fold_X_train = curr_fold['train'][all_features]
    curr_fold_X_train_scaled= curr_fold_X_train
    curr_fold_X_train_scaled[features_to_scale] = scaler.fit_transform(curr_fold_X_train[features_to_scale])
    curr_fold_y_train = curr_fold['train']['salary'] 
    
    tr_st = time.time()
    curr_model = model['modelobj'].fit(curr_fold_X_train_scaled, curr_fold_y_train)
    tr_time = time.time() - tr_st
    
    
    #Test
    curr_fold_X_test = curr_fold['test'][all_features]
    curr_fold_X_test_scaled = curr_fold_X_test
    curr_fold_X_test_scaled[features_to_scale] = scaler.fit_transform(curr_fold_X_test[features_to_scale])
    curr_fold_y_test = curr_fold['test']['salary']
    
    tst_st = time.time()
    curr_test_predict = curr_model.predict(curr_fold_X_test_scaled)
    tst_time = time.time() - tst_st
    
    #Compute Cost (MSE/RMSE)
    curr_fold_mse = MSE(curr_test_predict, curr_fold_y_test)
    curr_fold_rmse = curr_fold_mse**0.5
    
    return curr_fold_mse, curr_fold_rmse, tr_time, tst_time
    

def GetBestParams(m, model, alldata, all_features, features_to_scale, k,\
                  scaler = preprocessing.StandardScaler()):
                  
    X_train = alldata[all_features]
    y_train = alldata['salary']
    X_train_scaled = X_train
    X_train_scaled[features_to_scale] = scaler.fit_transform(X_train[features_to_scale])
    
    
    if m in ['Ridge','DT']:
        model_search = GridSearchCV(model['modelobj'], model['params'], cv=k, verbose=0, n_jobs=-1,\
                                    scoring ='neg_mean_squared_error')
        model_search.fit(X_train_scaled, y_train)
        best_model = model_search.best_estimator_
        return best_model
    elif m in ['RF','GBR']:
        model_search = GridSearchCV(model['modelobj'], model['params'],verbose=0, n_jobs=-1,\
                                    scoring ='neg_mean_squared_error')
        model_search.fit(X_train_scaled, y_train)
        best_model = model_search.best_estimator_
        return best_model
    else:
        return model['modelobj']        


2.2.1 Run Grid Search on all models to get initial best performing parameters

In [17]:
#Get Model Params Using Grid Search CV

def RunExpOnAllData(models,all_features,features_to_scale, alldata, sc = preprocessing.StandardScaler()):
    for m in models:
        print(m,'Finding Best Params')
        models[m]['modelobj'] = GetBestParams(m, models[m], alldata, all_features,features_to_scale, 5, sc)
        print(m,'Best Params Found')
        


RunExpOnAllData(models, all_features,features_to_scale, train_full_encoded, scalermain)

LR Finding Best Params
LR Best Params Found
Ridge Finding Best Params
Ridge Best Params Found
DT Finding Best Params
DT Best Params Found


2.2.2  Evaluate all models to see what perfoms best on differnt folds created in 1.4

In [18]:
# Find model performance vased on k-fold data created earliers ( See 1.4)
import multiprocessing as mp



def RunExpOnSplitData(kfolddata, models, all_features,features_to_scale, sc = preprocessing.StandardScaler()):
    for m in models:
        pool = mp.Pool(mp.cpu_count() -1 )
        #mse_rmse_results = pool.starmap(EvalModel, [ (models[m], kfolddata[curr_fold], train_features) for curr_fold in kfolddata])
        mse_rmse_results = pool.starmap_async(EvalModelSplitData, [(m, models[m], kfolddata[curr_fold], all_features,features_to_scale, sc) for curr_fold in kfolddata]).get()
        pool.close()
    
        models[m]['mse'] = sum([i[0] for i in mse_rmse_results])/len(mse_rmse_results)
        models[m]['rmse'] = sum([i[1] for i in mse_rmse_results])/len(mse_rmse_results)
        models[m]['tr_time'] = sum([i[2] for i in mse_rmse_results])/len(mse_rmse_results)
        models[m]['tst_time'] = sum([i[3] for i in mse_rmse_results])/len(mse_rmse_results)
        
        #print(m, models[m]['mse'], models[m]['rmse'],models[m]['tr_time'],models[m]['tst_time'], k)              

k = 10
kfolddata =  GetKFoldData(train_full_encoded, k)    
RunExpOnSplitData(kfolddata, models, all_features,features_to_scale, scalermain)
models_df =  pd.DataFrame.from_dict(models, orient='index')
models_df[['mse','rmse','tr_time','tst_time']]

,mse,rmse,tr_time,tst_time
DT,422.108463,20.545234,15.409462,0.062072
LR,394.543680,19.863039,5.450826,0.094322
Ridge,394.539984,19.862946,1.413677,0.056092


2.2.2  Choose the model to be used

In [19]:
#Choose Model Having the least RMSE on average over k-fold data
modelchosen = ('LR',models['LR'])
for m in models:
    if models[m]['rmse']<modelchosen[1]['rmse']:
        modelchosen = (m,models[m])

print(modelchosen[0], ' is the model choosen')

Ridge  is the model choosen


In [20]:

#modelchosen = ('LR',models['LR'])


In [21]:
# COmmented

# curr_model_mse = 0 
# curr_model_rmse = 0                 
#         for curr_fold in kfolddata:
#             curr_fold_mse, curr_fold_rmse = EvalModel(models[m], kfolddata[curr_fold], train_features)
#             print(m, curr_fold_mse, curr_fold_rmse)
#             curr_model_mse += curr_fold_mse
#             curr_model_rmse += curr_fold_rmse

#         models[m]['mse']  = curr_model_mse/k
#         models[m]['rmse'] = curr_model_rmse/k
#         print(m, models[m]['mse'], models[m]['rmse'], k)
#         print()

# 3. Feature Importance w.r.t best Model

For the best performing model I have tried to measure feature importance based on the impact they cause when the feature has been removed. (I have removed the features from the original category and not the encoded ones)

In [22]:
#Features whose importance will be evaluated
#org_features = ['companyId', 'jobType', 'degree', 'major', 'industry','yearsExperience', 'milesFromMetropolis']
#train_features
org_features = org_features[1:]
org_features

['companyId',
 'jobType',
 'degree',
 'major',
 'industry',
 'yearsExperience',
 'milesFromMetropolis']

In [23]:
#Check change in RMSE performance on k-fold data when a feature is excluded
feature_imp_results = {}
#for m in models:
#    modelchosen = (m, models[m])
for f in org_features:
    try:

        all_features_wo_curr_f = [of for of in all_features if f not in of]
        features_to_scale_wo_curr_f = [of for of in features_to_scale if f not in of]         

        pool = mp.Pool(mp.cpu_count() -1 )
        mse_rmse_results = pool.starmap_async(EvalModelSplitData, [(modelchosen[0], modelchosen[1], kfolddata[curr_fold], all_features_wo_curr_f, features_to_scale_wo_curr_f, scalermain) \
                                                                   for curr_fold in kfolddata]).get()
        pool.close()
        if f not in feature_imp_results:
            feature_imp_results[f] = {}
            feature_imp_results[f]['Model'] = modelchosen[0]
            feature_imp_results[f]['mse'] = sum([i[0] for i in mse_rmse_results])/len(mse_rmse_results)
            feature_imp_results[f]['rmse'] = sum([i[1] for i in mse_rmse_results])/len(mse_rmse_results)
            feature_imp_results[f]['percentage_error_increase'] = 100 * (feature_imp_results[f]['rmse'] - modelchosen[1]['rmse'])/modelchosen[1]['rmse']
    except:
        print('issued with feature', f)


In [24]:
feature_imp_results_df = pd.DataFrame.from_dict(feature_imp_results, orient='index')
feature_imp_results_df.sort_values(by='percentage_error_increase', ascending=False)

,Model,mse,rmse,percentage_error_increase
jobType,Ridge,774.503132,27.829794,40.109096
yearsExperience,Ridge,604.822658,24.593027,23.813590
milesFromMetropolis,Ridge,527.651915,22.970576,15.645365
industry,Ridge,524.679292,22.905781,15.319152
degree,Ridge,414.584386,20.361277,2.508846
major,Ridge,411.528611,20.286092,2.130327
companyId,Ridge,394.510556,19.862205,-0.003728


In [25]:
# feature_imp = feature_imp_results_df.groupby('Model')
# feature_imp.sort_values(by='percentage_error_increase', ascending=False)

In [26]:
# import dill
# dill.dump_session('/media/oldmonk/D/submission_notebook_env_before_end_Section3.db')

# 4. Predict Salaries for Test Data